In [ ]:
import sys, os
project_root = os.getcwd()
sys.path.append(project_root[:-6])

import numpy as np
from quadtree_grid import build_quadtree_grid_optimized
from deformed_grid import build_deformed_grid
from region_division.obstacles.mountain_generator import generate_mountains
from region_division.obstacles.maze_generator import render_maze, generate_maze
from visualization import (
    visualize_3d_grid, 
    visualize_grid_layer, 
    save_figure_as_png, 
    display_all_visualizations
)

def main():
    # Размеры карты
    # width, height = 100, 100
    # depth = 20
    
    # print("Генерация карты местности...")
    
    # # Генерация горной местности
    # mountain_map = generate_mountains(5, (width, height, depth))
    
    # # Генерация лабиринта
    # removed_walls = generate_maze(10, 10, add_connections=5)
    # maze_map = render_maze(removed_walls, 10, 10, 5, 2, 15)
    
    # # Комбинация обоих типов препятствий
    # combined_map = np.zeros_like(mountain_map)
    # combined_map[:maze_map.shape[0], :maze_map.shape[1], :maze_map.shape[2]] = maze_map
    # combined_map = np.logical_or(combined_map, mountain_map).astype(np.int8)
    
    map_size = (50, 50, 20)
    width, height = 50, 50
    depth = 20
    #obstacles = generate_mountains(15, map_size)
    delete_walls = generate_maze(10, 10, 4)
    obstacles = render_maze(delete_walls, 12, 12, 2, 2, 19)
    # Используем комбинированную карту
    grid_3d = obstacles
    
    print("Генерация сетки методом квадродеревьев...")
    nodes_edges_per_layer = []
    
    for z in range(depth):
        grid_slice = grid_3d[z]
        nodes_2d, edges_2d = build_quadtree_grid_optimized(grid_slice)
        
        # Добавляем Z-координату
        if len(nodes_2d) > 0:
            nodes_3d = np.column_stack((nodes_2d, np.full(len(nodes_2d), z)))
            nodes_edges_per_layer.append((nodes_3d, edges_2d, z))
    
    # Объединяем узлы и ребра из всех слоев
    all_nodes_quadtree = []
    all_edges_quadtree = []
    node_offset = 0
    
    for nodes_3d, edges_2d, z in nodes_edges_per_layer:
        all_nodes_quadtree.extend(nodes_3d)
        
        # Обновляем индексы ребер
        if len(edges_2d) > 0:
            updated_edges = edges_2d + node_offset
            all_edges_quadtree.extend(updated_edges)
        
        node_offset += len(nodes_3d)
    
    all_nodes_quadtree = np.array(all_nodes_quadtree)
    all_edges_quadtree = np.array(all_edges_quadtree)
    
    print("Генерация сетки методом деформации...")
    all_nodes_deformed, all_edges_deformed = build_deformed_grid(grid_3d)
    
    # Визуализация результатов
    print("Создание визуализации...")
    
    # Словарь для хранения всех фигур
    figures = {}
    
    # Квадродерево 3D
    figures["квадродерево_3d"] = visualize_3d_grid(
        all_nodes_quadtree, 
        all_edges_quadtree, 
        grid_3d, 
        title="3D сетка на основе квадродеревьев"
    )
    
    # Деформация 3D
    figures["деформация_3d"] = visualize_3d_grid(
        all_nodes_deformed, 
        all_edges_deformed, 
        grid_3d, 
        title="3D сетка на основе деформации"
    )
    
    # Визуализация срезов
    middle_layer = depth // 2
    
    # Срез квадродерева
    figures["квадродерево_слой"] = visualize_grid_layer(
        all_nodes_quadtree, 
        all_edges_quadtree,
        grid_3d[middle_layer], 
        middle_layer, 
        title="Слой сетки (Квадродерево)"
    )
    
    # Срез деформированной сетки
    figures["деформация_слой"] = visualize_grid_layer(
        all_nodes_deformed, 
        all_edges_deformed,
        grid_3d[middle_layer], 
        middle_layer, 
        title="Слой сетки (Деформация)"
    )
    
    # Сохраняем HTML-файлы
    print("Сохранение HTML-файлов...")
    for name, fig in figures.items():
        if fig is not None:
            fig.write_html(f"{name}.html")
    
    # Сохраняем PNG-файлы
    print("Сохранение PNG-файлов...")
    for name, fig in figures.items():
        if fig is not None:
            save_figure_as_png(fig, name)
    
    # Отображаем визуализации
    print("Отображение визуализаций...")
    display_all_visualizations(figures)
    
    print("Готово! Результаты сохранены в HTML и PNG файлы.")

if __name__ == "__main__":
    main()